In [1]:
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, matthews_corrcoef, precision_recall_curve, auc
import seaborn as sns,matplotlib.pyplot as plt

from library.utils import evaluate, read_data

In [2]:
SHORT = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']

In [3]:
X,y_noisy,y_real = read_data(SHORT[0])

noise:0.0845442536327609, imb:8.463,80,677, Shape:(757, 65)


In [ ]:
clf = RandomForestClassifier().fit(X,y_real)
yp = clf.predict_proba(X)

In [ ]:
pr,rec,thr = precision_recall_curve(y_real,yp[:,1])
pr,rec,thr

In [ ]:
auc(rec,pr)

In [ ]:
plt.plot(rec,pr);